### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [1]:
%pip install semantic-link-labs

StatementMeta(, 693b1b8e-5cf3-4a12-9aa5-72b35632d0f4, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.5/635.5 kB 11.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.8.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-2370f380-58ec-4a5f-ba95-da8d31bc47b3
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
%pip show semantic-link-sempy
print('\n')
%pip show semantic-link-labs

StatementMeta(, 06706660-e2f6-474c-a602-e0e6958be0ba, 9, Finished, Available, Finished)

Name: semantic-link-sempy
Version: 0.9.0
Summary: Semantic link for Microsoft Fabric
Home-page: https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview
Author: Microsoft
Author-email: semanticdatascience@service.microsoft.com
License: proprietary and confidential
Location: /nfs4/pyenv-b41b750f-140e-429f-9466-378a7148a4d1/lib/python3.11/site-packages
Requires: clr-loader, graphviz, pyarrow, pythonnet, rich, scikit-learn, setuptools, tqdm
Required-by: semantic-link-labs
Note: you may need to restart the kernel to use updated packages.


Name: semantic-link-labs
Version: 0.9.1
Summary: Semantic Link Labs for Microsoft Fabric
Home-page: 
Author: Microsoft Corporation
Author-email: 
License: MIT License
Location: /nfs4/pyenv-b41b750f-140e-429f-9466-378a7148a4d1/lib/python3.11/site-packages
Requires: anytree, jsonpath_ng, polib, powerbiclient, semantic-link-sempy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### 2. Import the library and set initial parameters

In [2]:
import sempy_labs as labs
from sempy_labs import migration, directlake
import sempy_labs.report as rep

dataset_name = 'Import_Contoso10K' #Enter the import/DQ semantic model name
workspace_name = 'FS25_DirectLakeMigration' #Enter the workspace of the import/DQ semantic model. It set to none it will use the current workspace.
new_dataset_name = 'DL_Contoso10K' #Enter the new Direct Lake semantic model name
new_dataset_workspace_name = 'FS25_DirectLakeMigration' #Enter the workspace where the Direct Lake model will be created. If set to None it will use the current workspace.
lakehouse_name = 'SummitLake' #Enter the lakehouse to be used for the Direct Lake model. If set to None it will use the lakehouse attached to the notebook.
lakehouse_workspace_name = 'FS25_DirectLakeMigration' #Enter the lakehouse workspace. If set to None it will use the new_dataset_workspace_name.

StatementMeta(, 693b1b8e-5cf3-4a12-9aa5-72b35632d0f4, 9, Finished, Available, Finished)

### 3. Create the [Power Query Template](https://learn.microsoft.com/power-query/power-query-template) file

This encapsulates all of the semantic model's Power Query logic into a single file.

In [4]:
migration.create_pqt_file(dataset = dataset_name, workspace = workspace_name)

StatementMeta(, 06706660-e2f6-474c-a602-e0e6958be0ba, 11, Finished, Available, Finished)

🟢 'PowerQueryTemplate.pqt' has been created based on the 'Import_Contoso10K' semantic model in the 'FS25_DirectLakeMigration' workspace within the Files section of your lakehouse.


### 4. Import the Power Query Template to Dataflows Gen2

- Open the [OneLake file explorer](https://www.microsoft.com/download/details.aspx?id=105222) and sync your files (right click -> Sync from OneLake)

- Navigate to your lakehouse. From this window, create a new Dataflows Gen2 and import the Power Query Template file from OneLake (OneLake -> Workspace -> Lakehouse -> Files...), and publish the Dataflows Gen2.

<div class="alert alert-block alert-info">
<b>Important!</b> Make sure to create the Dataflows Gen2 from within the lakehouse window. That will ensure that all the tables automatically map to that lakehouse as the destination. Otherwise, you will have to manually map each table to its destination individually.
</div>

### 5. Create the Direct Lake model based on the import/DQ semantic model

Calculated columns are not migrated to the Direct Lake model as they are not supported in Direct Lake mode.

In [3]:
import time
labs.create_blank_semantic_model(dataset = new_dataset_name, workspace = new_dataset_workspace_name, overwrite=True)

migration.migrate_calc_tables_to_lakehouse(
    dataset = dataset_name,
    new_dataset = new_dataset_name,
    workspace = workspace_name,
    new_dataset_workspace = new_dataset_workspace_name,
    lakehouse = lakehouse_name,
    lakehouse_workspace = lakehouse_workspace_name
)
migration.migrate_tables_columns_to_semantic_model(
    dataset = dataset_name,
    new_dataset = new_dataset_name,
    workspace = workspace_name,
    new_dataset_workspace = new_dataset_workspace_name,
    lakehouse = lakehouse_name,
    lakehouse_workspace = lakehouse_workspace_name
)
migration.migrate_calc_tables_to_semantic_model(
    dataset = dataset_name,
    new_dataset = new_dataset_name,
    workspace = workspace_name,
    new_dataset_workspace = new_dataset_workspace_name,
    lakehouse = lakehouse_name,
    lakehouse_workspace = lakehouse_workspace_name
)
migration.migrate_model_objects_to_semantic_model(
    dataset = dataset_name,
    new_dataset = new_dataset_name,
    workspace = workspace_name,
    new_dataset_workspace = new_dataset_workspace_name
)
migration.migrate_field_parameters(
    dataset = dataset_name,
    new_dataset = new_dataset_name,
    workspace = workspace_name,
    new_dataset_workspace = new_dataset_workspace_name
)
time.sleep(2)
labs.refresh_semantic_model(dataset = new_dataset_name, workspace = new_dataset_workspace_name)
migration.refresh_calc_tables(dataset = new_dataset_name, workspace = new_dataset_workspace_name)
labs.refresh_semantic_model(dataset = new_dataset_name, workspace = new_dataset_workspace_name)

StatementMeta(, f1c50e73-895e-42e8-a9b4-0e3cdf2a1af3, 10, Finished, Available, Finished)

🟢 The 'DL_Contoso10K' semantic model was created within the 'FS25_DirectLakeMigration' workspace.
🟡 The 'Import_Contoso10K' semantic model in the 'FS25_DirectLakeMigration' workspace has no calculated tables.
⌛ Updating 'DL_Contoso10K' based on 'Import_Contoso10K'...
🟢 The 'DatabaseQuery' expression has been added.
🟢 The 'Product' table has been added.
🟢 The 'Sales' table has been added.
🟢 The 'Store' table has been added.
🟢 The 'Date' table has been added.
🟢 The 'Customer' table has been added.
🟢 The 'Product'[ProductKey] column has been added.
🟢 The 'Product'[Product Code] column has been added.
🟢 The 'Product'[Product Name] column has been added.
🟢 The 'Product'[Manufacturer] column has been added.
🟢 The 'Product'[Brand] column has been added.
🟢 The 'Product'[Color] column has been added.
🟢 The 'Product'[Weight Unit Measure] column has been added.
🟢 The 'Product'[Weight] column has been added.
🟢 The 'Product'[Unit Cost] column has been added.
🟢 The 'Product'[Unit Price] column has b

### 6. Show migrated/unmigrated objects

In [4]:
# Changed so output goes to DataFrame 
df = migration.migration_validation(
    dataset = dataset_name,
    new_dataset = new_dataset_name, 
    workspace = workspace_name, 
    new_dataset_workspace = new_dataset_workspace_name
)

display(df)

StatementMeta(, f1c50e73-895e-42e8-a9b4-0e3cdf2a1af3, 11, Finished, Available, Finished)

94.0% migrated


SynapseWidget(Synapse.DataFrame, c0ea5710-8bf3-451e-a968-982dd5b5fd99)

### Rebind all reports using the old semantic model to the new Direct Lake semantic model

In [ ]:
#rep.report_rebind_all(
#    dataset = dataset_name,
#    dataset_workspace = workspace_name,
#    new_dataset = new_dataset_name,
#    new_dataset_workpace = new_dataset_workspace_name,
#    report_workspace = None
#)

### 7. Rebind reports one-by-one (optional)

In [4]:
report_name = 'DL_Contoso10KReport' # Enter report name which you want to rebind to the new Direct Lake model

rep.report_rebind(
    report = report_name,
    dataset = new_dataset_name,
    report_workspace=workspace_name,
    dataset_workspace = new_dataset_workspace_name)

StatementMeta(, 693b1b8e-5cf3-4a12-9aa5-72b35632d0f4, 11, Finished, Available, Finished)

🟢 The 'DL_Contoso10KReport' report has been successfully rebinded to the 'DL_Contoso10K' semantic model.


### 8. Show unsupported objects

In [6]:
dfT, dfC, dfR = directlake.show_unsupported_direct_lake_objects(dataset = dataset_name, workspace = workspace_name)

print('Calculated Tables are not supported...')
display(dfT)
print("Learn more about Direct Lake limitations here: https://learn.microsoft.com/power-bi/enterprise/directlake-overview#known-issues-and-limitations")
print('Calculated columns are not supported. Columns of binary data type are not supported.')
display(dfC)
print('Columns used for relationship must be of the same data type.')
display(dfR)

StatementMeta(, f1c50e73-895e-42e8-a9b4-0e3cdf2a1af3, 13, Finished, Available, Finished)

Calculated Tables are not supported...


SynapseWidget(Synapse.DataFrame, 698fed1b-4de9-4fd1-8cc7-9299ecce92c0)

Learn more about Direct Lake limitations here: https://learn.microsoft.com/power-bi/enterprise/directlake-overview#known-issues-and-limitations
Calculated columns are not supported. Columns of binary data type are not supported.


SynapseWidget(Synapse.DataFrame, 2919252f-3fa1-4cb6-88eb-36a5d2c58d2e)

Columns used for relationship must be of the same data type.


SynapseWidget(Synapse.DataFrame, d6f6a074-2328-47d4-b1c4-2fe13bc828c1)

### 9. Schema check between semantic model tables/columns and lakehouse tables/columns

This will list any tables/columns which are in the new semantic model but do not exist in the lakehouse

In [7]:
directlake.direct_lake_schema_compare(dataset = new_dataset_name, workspace = new_dataset_workspace_name)

StatementMeta(, f1c50e73-895e-42e8-a9b4-0e3cdf2a1af3, 14, Finished, Available, Finished)

🟢 All tables exist in the 'SummitLake' lakehouse within the 'FS25_DirectLakeMigration' workspace.
🟢 All columns exist in the 'SummitLake' lakehouse within the 'FS25_DirectLakeMigration' workspace.


### 10. Show calculated tables which have been migrated to the Direct Lake semantic model as regular tables

In [ ]:
directlake.list_direct_lake_model_calc_tables(dataset = new_dataset_name, workspace = new_dataset_workspace_name)

## 11. Display info about LakeHouse Tables

In [8]:
df = labs.lakehouse.get_lakehouse_tables(lakehouse = 'SummitLake', workspace= 'FS25_DirectLakeMigration', extended='true', count_rows = 'true')

display(df)

StatementMeta(, f1c50e73-895e-42e8-a9b4-0e3cdf2a1af3, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 75ebf5c8-73c3-4a2e-9824-c4468358da1a)